    Duncan Rocha
    3/22/18
    STELA Collective

    Example code for how one might use the server to send commands to the arduino.

In [15]:
import numpy as np
import json
import time
import urllib2

In [87]:

class server:
    def __init__(self):
        # Specify flask ip adress here (this one works if flask is run on your own computer)
        self.url = 'http://127.0.0.1:5123'
        
    def set_targ(self,az_alt):
        # Set the target position in the arduino (notice dictionary protocol)
        data = {'targ': az_alt}
        req = urllib2.Request(self.url+'/set_targ')
        req.add_header('Content-Type', 'application/json')
        response = urllib2.urlopen(req, json.dumps(data))
        
    def get_pos(self):
        # Get the current position of the arduino (again, protocols)
        a = urllib2.urlopen(self.url+'/get_pos')
        dic = json.loads(a.read())
        az_alt = dic['pos']
        az_alt_targ = dic['targ']
        return az_alt,az_alt_targ
    
    def setup(self):
        # Currently nonfuntional
        a = urllib2.urlopen(self.url+'/setup')
        return json.loads(a.read())

    def move(self,az_alt):
        # move a small incriment
        data = {'move': az_alt}
        req = urllib2.Request(self.url+'/move')
        req.add_header('Content-Type', 'application/json')

        response = urllib2.urlopen(req, json.dumps(data))

    
#serv = server()
#serv.set_targ([20,10])
#serv.get_pos()
#print serv.setup()
serv.move([10,10])
serv.move([10,10])
print serv.get_pos()

([0.0, 0.0], [10.0, 10.0])
